## 2 Data Cleaning & EDA

### 2.1 Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')



### 2.2 Load data

In [2]:
# Load the raw data
df = pd.read_csv('../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv')
print(f"Data loaded: {df.shape}")

Data loaded: (7043, 21)


### 2.3 Handle Missing Values

In [3]:
# Fix the TotalCharges issue
# The problem: TotalCharges is stored as 'object' (text) instead of 'float64' (number)
# This is because some values are actually spaces/blanks

print("Before cleaning:")
print(f"TotalCharges dtype: {df['TotalCharges'].dtype}")
print(f"Missing values in TotalCharges: {df['TotalCharges'].isnull().sum()}")

# SOLUTION: Convert to numeric, treating blanks as NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

print(f"\nAfter conversion to numeric:")
print(f"TotalCharges dtype: {df['TotalCharges'].dtype}")
print(f"Missing values in TotalCharges: {df['TotalCharges'].isnull().sum()}")

# Remove rows with missing TotalCharges
initial_rows = len(df)
df = df.dropna(subset=['TotalCharges'])
print(f"\nRows removed: {initial_rows - len(df)}")
print(f"Remaining rows: {len(df)}")

# WHAT THIS DOES:
# - Converts text numbers to actual numbers
# - Removes row with missing value
# - Ensures data integrity

Before cleaning:
TotalCharges dtype: object
Missing values in TotalCharges: 0

After conversion to numeric:
TotalCharges dtype: float64
Missing values in TotalCharges: 11

Rows removed: 11
Remaining rows: 7032


### 2.4 Remove Unnecessary Columns

In [4]:
# Remove columns that don't help prediction
# customerID is unique for each customer - not useful for modeling

print("Columns before removal:")
print(df.columns.tolist())

# Remove customerID (unique identifier, not predictive)
df = df.drop('customerID', axis=1)

print(f"\nColumns after removal: {len(df.columns)}")
print(df.columns.tolist())

# WHAT THIS DOES:
# - Removes non-predictive columns
# - Reduces data dimensionality

Columns before removal:
['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

Columns after removal: 20
['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']


- Columns before: 21
- Columns after: 20

### 2.5 Encode Target Variable

In [5]:
# Convert 'Yes'/'No' to 1/0 for machine learning

print("Churn values before encoding:")
print(df['Churn'].unique())


# Map Yes/No to 1/0

df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

print("\nChurn values after encoding:")
print(df['Churn'].unique())

print(f"\nChurn Distribution:")
print(df['Churn'].value_counts())

# WHAT THIS DOES:
# - Converts categorical target to binary (1/0)
# - Makes it compatible with machine learning algorithms

Churn values before encoding:
['No' 'Yes']

Churn values after encoding:
[0 1]

Churn Distribution:
Churn
0    5163
1    1869
Name: count, dtype: int64


In [6]:
# Churn distribution
churn_counts = (
    df['Churn']
    .value_counts()
    .reindex([0, 1])
    .reset_index()
)

churn_counts.columns = ['Churn', 'Count']

# Map labels
churn_counts['Churn Label'] = churn_counts['Churn'].map({
    0: 'No Churn',
    1: 'Churn'
})


In [7]:
# Calculate percentages
churn_counts['Percentage'] = (
    churn_counts['Count'] / churn_counts['Count'].sum() * 100
).round(2)


In [8]:
# Visualize churn distribution

fig = px.bar(
    churn_counts,
    x='Churn Label',
    y='Count',
    text=churn_counts['Percentage'].astype(str) + '%',
    title='Customer Churn Distribution After Encoding'
)

fig.update_traces(textposition='outside')

fig.update_layout(
    xaxis_title='Churn Status',
    yaxis_title='Number of Customers',
    template='plotly_white',
    uniformtext_minsize=12,
    uniformtext_mode='hide'
)
fig.write_image(
    "../data/processed/custom_churn_distribution.png",
    scale=3  
)
fig.show()


### 2.6 Separate Features by Type

In [10]:
# Identify categorical vs numerical features
# This helps us know how to process each type differently

print("Feature Classification:")


numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

print(f"\nNumerical Features ({len(numerical_features)}):")
for col in numerical_features:
    print(f"  - {col}")

print(f"\nCategorical Features ({len(categorical_features)}):")
for col in categorical_features:
    print(f"  - {col}")

# Store for later use
print(f"\n Total Features: {len(numerical_features) + len(categorical_features)}")

# WHAT THIS DOES:
# - Organizes features by data type
# - Helps plan data processing strategy

Feature Classification:

Numerical Features (5):
  - SeniorCitizen
  - tenure
  - MonthlyCharges
  - TotalCharges
  - Churn

Categorical Features (15):
  - gender
  - Partner
  - Dependents
  - PhoneService
  - MultipleLines
  - InternetService
  - OnlineSecurity
  - OnlineBackup
  - DeviceProtection
  - TechSupport
  - StreamingTV
  - StreamingMovies
  - Contract
  - PaperlessBilling
  - PaymentMethod

 Total Features: 20


### 2.7  Examine Categorical Features

In [11]:
# Look at distribution of categorical features

print("Categorical Features Distribution:")


for col in categorical_features:
    print(f"\n{col}:")
    print(df[col].value_counts())
    print(f"  Unique values: {df[col].nunique()}")

# WHAT THIS DOES:
# - Shows how many of each category we have
# - Helps identify imbalanced categories

Categorical Features Distribution:

gender:
gender
Male      3549
Female    3483
Name: count, dtype: int64
  Unique values: 2

Partner:
Partner
No     3639
Yes    3393
Name: count, dtype: int64
  Unique values: 2

Dependents:
Dependents
No     4933
Yes    2099
Name: count, dtype: int64
  Unique values: 2

PhoneService:
PhoneService
Yes    6352
No      680
Name: count, dtype: int64
  Unique values: 2

MultipleLines:
MultipleLines
No                  3385
Yes                 2967
No phone service     680
Name: count, dtype: int64
  Unique values: 3

InternetService:
InternetService
Fiber optic    3096
DSL            2416
No             1520
Name: count, dtype: int64
  Unique values: 3

OnlineSecurity:
OnlineSecurity
No                     3497
Yes                    2015
No internet service    1520
Name: count, dtype: int64
  Unique values: 3

OnlineBackup:
OnlineBackup
No                     3087
Yes                    2425
No internet service    1520
Name: count, dtype: int64
  Unique 

### 2.8 Generate Pandas Profiling Report

In [12]:
# Create an automated EDA report
# This takes a few minutes to run but generates comprehensive statistics

from ydata_profiling import ProfileReport


print("Generating Pandas Profiling Report...")
print("This may take 2-3 minutes...")

profile = ProfileReport(df, title="Telco Customer Churn EDA Report")
profile.to_file("../data/processed/eda_report.html")

print("EDA Report generated!")
print("View at: data/processed/eda_report.html")

# WHAT THIS DOES:
# - Generates comprehensive statistics
# - Identifies correlations and missing values
# - Creates interactive HTML report
# - Great for quick EDA overview

Generating Pandas Profiling Report...
This may take 2-3 minutes...


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 164.21it/s]

EDA Report generated!
View at: data/processed/eda_report.html


### 2.9 Correlation Analysis - Numerical Features

In [13]:
type(df[numerical_features + ['Churn']].corr()['Churn'])


pandas.core.frame.DataFrame

In [14]:
df.columns[df.columns == 'Churn']


Index(['Churn'], dtype='object')

In [15]:
type(df.columns)


pandas.core.indexes.base.Index

In [16]:
df = df.loc[:, ~df.columns.duplicated()]
df['Churn'] = df['Churn'].astype(int)


In [17]:
# Find relationships between numerical features and churn

print("Correlation with Churn:\n")

# Step 1: Correlation matrix
corr_matrix = df[numerical_features + ['Churn']].corr()

# Step 2: Extract Churn column safely
corr_with_churn = corr_matrix.loc[:, 'Churn']

# If duplicate Churn columns exist, reduce to Series
if isinstance(corr_with_churn, pd.DataFrame):
    corr_with_churn = corr_with_churn.iloc[:, 0]

# Step 3: Sort
corr_with_churn = corr_with_churn.sort_values(ascending=False)

def interpret_corr(feature, value):
    if feature == 'Churn':
        return ""
    if value >= 0.3:
        return "✓ Strong positive (higher values → more churn)"
    elif value >= 0.1:
        return "✓ Weak positive (higher values → more churn)"
    elif value <= -0.3:
        return "⚠️ Strong negative (higher values → less churn)"
    elif value < 0:
        return "✓ Negative (higher values → less churn)"
    else:
        return "• Very weak / no clear relationship"

# Step 4: Pretty print
for feature, value in corr_with_churn.items():
    print(f"{feature:<18} {value:>8.6f}  {interpret_corr(feature, value)}")


Correlation with Churn:

Churn              1.000000  
Churn              1.000000  
MonthlyCharges     0.192858  ✓ Weak positive (higher values → more churn)
SeniorCitizen      0.150541  ✓ Weak positive (higher values → more churn)
TotalCharges       -0.199484  ✓ Negative (higher values → less churn)
tenure             -0.354049  ⚠️ Strong negative (higher values → less churn)


Key Insights:

- tenure: -0.352 → Strong negative correlation (longer tenure = less churn)
- MonthlyCharges: 0.194 → Weak positive (higher charges = more churn)
- SeniorCitizen: 0.150 → Weak positive (seniors churn more)

### 2.10 Visualization - Correlation Heatmap

In [18]:
# Create heatmap showing correlations visually
# This helps us see relationships at a glance

fig, ax = plt.subplots(figsize=(10, 8))

# Create correlation matrix for numerical features
corr_matrix = df[numerical_features + ['Churn']].corr()

# Create heatmap
sns.heatmap(corr_matrix, 
            annot=True,           # Show values
            fmt='.3f',            # 3 decimal places
            cmap='coolwarm',      # Color scheme
            center=0,             # Center at 0
            square=True,          # Square cells
            linewidths=0.5,
            cbar_kws={'label': 'Correlation Coefficient'},
            ax=ax)

ax.set_title('Numerical Features Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../data/processed/03_correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()


# WHAT THIS DOES:
# - Red = positive correlation
# - Blue = negative correlation
# - Intensity = strength of correlation

- Heatmap showing tenure has strong negative correlation (dark blue) with churn
- Monthly charges shows weak positive correlation (light red)

Churn  Tenure  Monthly  Total  Senior
Churn    1.00   -0.35    0.19   -0.20   0.15
Tenure  -0.35    1.00    0.25   0.82  -0.04
...add  more

Colors:
🔵 Dark Blue  = Strong negative (-1)
⚪ White      = No correlation (0)
🔴 Dark Red   = Strong positive (+1)

### 2.11 Tenure Analysis

In [19]:
# Deep dive into tenure (months as customer)
# This is a key predictor of churn

print("Tenure Analysis:")

print(df['tenure'].describe())

# Compare tenure for churned vs non-churned customers
print("\nTenure by Churn Status:")
print(df.groupby('Churn')['tenure'].describe())

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution by churn
df[df['Churn']==0]['tenure'].hist(bins=30, ax=axes[0], color='#2ecc71', alpha=0.7, label='No Churn')
df[df['Churn']==1]['tenure'].hist(bins=30, ax=axes[0], color='#e74c3c', alpha=0.7, label='Churn')
axes[0].set_xlabel('Tenure (months)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Tenure Distribution by Churn Status')
axes[0].legend()

# Box plot
df.boxplot(column='tenure', by='Churn', ax=axes[1])
axes[1].set_xlabel('Churn Status (0=No, 1=Yes)')
axes[1].set_ylabel('Tenure (months)')
axes[1].set_title('Tenure Comparison by Churn')
plt.suptitle('')  # Remove automatic title

plt.tight_layout()
plt.savefig('../data/processed/04_tenure_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n Tenure analysis visualization saved!")

# WHAT THIS DOES:
# - Shows tenure distribution is different for churned customers
# - Churned customers have much lower tenure (left early)

Tenure Analysis:
count    7032.000000
mean       32.421786
std        24.545260
min         1.000000
25%         9.000000
50%        29.000000
75%        55.000000
max        72.000000
Name: tenure, dtype: float64

Tenure by Churn Status:
        count       mean        std  min   25%   50%   75%   max
Churn                                                           
0      5163.0  37.650010  24.076940  1.0  15.0  38.0  61.0  72.0
1      1869.0  17.979133  19.531123  1.0   2.0  10.0  29.0  72.0

 Tenure analysis visualization saved!


Key Insight: Churned customers have 19.7 months LESS tenure on average!

### 2.12 Monthly Charges Analysis

In [20]:
# Analyze relationship between monthly charges and churn

print("Monthly Charges Analysis:")

print(df['MonthlyCharges'].describe())

print("\nMonthly Charges by Churn Status:")
print(df.groupby('Churn')['MonthlyCharges'].describe())

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution
df[df['Churn']==0]['MonthlyCharges'].hist(bins=30, ax=axes[0], color='#2ecc71', alpha=0.7, label='No Churn')
df[df['Churn']==1]['MonthlyCharges'].hist(bins=30, ax=axes[0], color='#e74c3c', alpha=0.7, label='Churn')
axes[0].set_xlabel('Monthly Charges ($)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Monthly Charges Distribution by Churn')
axes[0].legend()

# Box plot
df.boxplot(column='MonthlyCharges', by='Churn', ax=axes[1])
axes[1].set_xlabel('Churn Status (0=No, 1=Yes)')
axes[1].set_ylabel('Monthly Charges ($)')
axes[1].set_title('Monthly Charges Comparison')
plt.suptitle('')

plt.tight_layout()
plt.savefig('../data/processed/05_monthly_charges_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n Monthly charges analysis visualization saved!")

# WHAT THIS DOES:
# - Churned customers tend to have higher monthly charges
# - Suggests price sensitivity as churn factor

Monthly Charges Analysis:
count    7032.000000
mean       64.798208
std        30.085974
min        18.250000
25%        35.587500
50%        70.350000
75%        89.862500
max       118.750000
Name: MonthlyCharges, dtype: float64

Monthly Charges by Churn Status:
        count       mean        std    min    25%    50%     75%     max
Churn                                                                   
0      5163.0  61.307408  31.094557  18.25  25.10  64.45  88.475  118.75
1      1869.0  74.441332  24.666053  18.85  56.15  79.65  94.200  118.35

 Monthly charges analysis visualization saved!


Key Insight: Customers who churn pay $12 MORE per month on average!

### 2.13 Contract Type Analysis

In [21]:
# Analyze how contract type affects churn
# This should be a major churn predictor

print("CONTRACT TYPE ANALYSIS")


# Count by type
print("\nContract Distribution:")
print(df['Contract'].value_counts())

# Churn rate by contract
print("\nChurn Rate by Contract Type:")
contract_analysis = df.groupby('Contract').agg({
    'Churn': ['count', 'sum', 'mean']
}).round(4)
contract_analysis.columns = ['Total', 'Churned', 'Churn_Rate']
contract_analysis['Churn_Rate'] = contract_analysis['Churn_Rate'] * 100
print(contract_analysis)

# Visualizations
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart: Churn rate
contract_rates = (df.groupby('Contract')['Churn'].sum() / 
                  df.groupby('Contract')['Churn'].count() * 100)
contract_rates.plot(kind='bar', ax=axes[0], color=['#e74c3c', '#f39c12', '#2ecc71'])
axes[0].set_title('Churn Rate by Contract Type')
axes[0].set_xlabel('Contract Type')
axes[0].set_ylabel('Churn Rate (%)')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45)

# Add value labels
for i, v in enumerate(contract_rates.values):
    axes[0].text(i, v + 1.5, f'{v:.1f}%', ha='center', fontweight='bold')

# Stacked bar: Count
contract_counts = pd.crosstab(df['Contract'], df['Churn'])
contract_counts.plot(kind='bar', stacked=False, ax=axes[1],
                     color=['#2ecc71', '#e74c3c'])
axes[1].set_title('Customer Count by Contract & Churn')
axes[1].set_xlabel('Contract Type')
axes[1].set_ylabel('Count')
axes[1].legend(['No Churn', 'Churn'])
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45)

plt.tight_layout()
plt.savefig('../data/processed/06_contract_analysis.png', dpi=300)
plt.show()

CONTRACT TYPE ANALYSIS

Contract Distribution:
Contract
Month-to-month    3875
Two year          1685
One year          1472
Name: count, dtype: int64

Churn Rate by Contract Type:
                Total  Churned  Churn_Rate
Contract                                  
Month-to-month   3875     1655       42.71
One year         1472      166       11.28
Two year         1685       48        2.85


Key Insight: 
- Month-to-month customers are 15x more likely to churn!
- Month-to-month contracts have highest churn (~42.7%)
- 2-year contracts have lowest churn (~2.9%)

### 2.14 Internet Service Analysis

In [ ]:

# Internet service type analysis

print("INTERNET SERVICE ANALYSIS")

print("\nInternet Service Distribution:")

print(df['InternetService'].value_counts())

print("\nChurn Rate by Internet Service:")
internet_analysis = df.groupby('InternetService').agg({
    'Churn': ['count', 'sum', 'mean']
}).round(4)
internet_analysis.columns = ['Total', 'Churned', 'Churn_Rate']
internet_analysis['Churn_Rate'] = internet_analysis['Churn_Rate'] * 100
print(internet_analysis)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Churn rate bar chart
internet_rates = (df.groupby('InternetService')['Churn'].sum() /
                  df.groupby('InternetService')['Churn'].count() * 100)
internet_rates.plot(kind='bar', ax=axes[0], color=['#3498db', '#e74c3c', '#95a5a6'])
axes[0].set_title('Churn Rate by Internet Service Type')
axes[0].set_xlabel('Internet Service')
axes[0].set_ylabel('Churn Rate (%)')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45)

for i, v in enumerate(internet_rates.values):
    axes[0].text(i, v + 1.5, f'{v:.1f}%', ha='center', fontweight='bold')

# Count chart
internet_counts = pd.crosstab(df['InternetService'], df['Churn'])
internet_counts.plot(kind='bar', stacked=False, ax=axes[1],
                     color=['#2ecc71', '#e74c3c'])
axes[1].set_title('Customer Count by Internet Service')
axes[1].set_xlabel('Internet Service')
axes[1].set_ylabel('Count')
axes[1].legend(['No Churn', 'Churn'])
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45)

plt.tight_layout()
plt.savefig('../data/processed/07_internet_service_analysis.png', dpi=300)
plt.show()

INTERNET SERVICE ANALYSIS

Internet Service Distribution:
InternetService
Fiber optic    3096
DSL            2416
No             1520
Name: count, dtype: int64

Churn Rate by Internet Service:
                 Total  Churned  Churn_Rate
InternetService                            
DSL               2416      459       19.00
Fiber optic       3096     1297       41.89
No                1520      113        7.43


Key Insight: Fiber optic customers churn 2x more than DSL!

### 2.15 Save Cleaned Dataset

In [23]:
# Save the cleaned data for next notebooks
df.to_csv('../data/processed/telco_cleaned.csv', index=False)
print(f"Cleaned dataset saved: {df.shape}")
print("Location: data/processed/telco_cleaned.csv")

# Create version folder
import shutil
import os
os.makedirs('../data/v1', exist_ok=True)
shutil.copy('../data/processed/telco_cleaned.csv', '../data/v1/telco_cleaned_v1.csv')
print("Version 1 saved to: data/v1/telco_cleaned_v1.csv")

Cleaned dataset saved: (7032, 20)
Location: data/processed/telco_cleaned.csv
Version 1 saved to: data/v1/telco_cleaned_v1.csv
